# Prompt Template Patterns with Amazon Nova

This notebook demonstrates various prompt engineering patterns using LangChain templates.

## Setup

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
from langchain_amazon_nova import ChatAmazonNova
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from langchain_core.messages import AIMessage, HumanMessage

# Initialize the model
llm = ChatAmazonNova(model="nova-pro-v1", temperature=0.7)

## 1. Simple String Template

Basic template with variable substitution.

In [ ]:
template = ChatPromptTemplate.from_template(
    "You are an expert in {subject}. Explain {concept} in simple terms."
)

chain = template | llm
result = chain.invoke({"subject": "physics", "concept": "quantum entanglement"})

print(result.content)

## 2. Chat Template with System Message

Use tuples to specify different message roles.

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {role}. Keep responses {style}."),
        ("human", "{input}"),
    ]
)

chain = chat_template | llm
result = chain.invoke(
    {
        "role": "pirate",
        "style": "brief and in character",
        "input": "What's the weather like?",
    }
)

print(result.content)

## 3. Few-Shot Examples

Provide examples to guide the model's behavior.

In [ ]:
# Define examples
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "hot", "output": "cold"},
]

# Create example template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Create few-shot prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Combine into final prompt
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are providing antonyms."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt | llm
result = chain.invoke({"input": "light"})

print(f"Input: 'light'")
print(f"Output: {result.content}")

## 4. Message Placeholder for Chat History

Use `MessagesPlaceholder` to inject conversation history.

In [ ]:
prompt_with_history = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# Create chat history
history = [
    HumanMessage(content="My name is Alice"),
    AIMessage(content="Nice to meet you, Alice!"),
]

chain = prompt_with_history | llm
result = chain.invoke({"chat_history": history, "input": "What's my name?"})

print(result.content)

## 5. Multi-Variable Complex Template

Templates can have many variables and conditional logic.

In [ ]:
complex_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {profession} with {years} years of experience."),
        (
            "human",
            "Context: {context}\n\nQuestion: {question}\n\nPlease answer in {style} style.",
        ),
    ]
)

chain = complex_template | llm
result = chain.invoke(
    {
        "profession": "software architect",
        "years": 15,
        "context": "We're building a high-traffic e-commerce platform",
        "question": "Should we use microservices or monolith?",
        "style": "concise",
    }
)

print(result.content)

## 6. Partial Variables

Pre-fill some template variables.

In [ ]:
# Create template with partial
template = ChatPromptTemplate.from_template("Translate '{text}' to {language}")

# Partially fill the template
spanish_template = template.partial(language="Spanish")
french_template = template.partial(language="French")

# Now only need to provide text
spanish_chain = spanish_template | llm
french_chain = french_template | llm

text = "Hello, how are you?"

spanish = spanish_chain.invoke({"text": text})
french = french_chain.invoke({"text": text})

print(f"Spanish: {spanish.content}")
print(f"French: {french.content}")

## 7. Dynamic Few-Shot Selection

Select relevant examples based on input.

In [ ]:
# Library of examples
example_library = {
    "math": [
        {"input": "2+2", "output": "4"},
        {"input": "5*3", "output": "15"},
    ],
    "grammar": [
        {"input": "their there", "output": "they're"},
        {"input": "your you're", "output": "you're"},
    ],
}


def create_prompt_for_category(category: str):
    """Create a prompt with relevant examples."""
    examples = example_library.get(category, [])

    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{input}"),
            ("ai", "{output}"),
        ]
    )

    few_shot = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )

    return ChatPromptTemplate.from_messages(
        [
            ("system", f"You are helping with {category} problems."),
            few_shot,
            ("human", "{input}"),
        ]
    )


# Use math examples
math_prompt = create_prompt_for_category("math")
math_chain = math_prompt | llm

result = math_chain.invoke({"input": "10-3"})
print(f"Math result: {result.content}")

## 8. Template Composition

Build complex prompts from smaller reusable pieces.

In [ ]:
# Reusable components
system_template = "You are an AI assistant specialized in {domain}."
context_template = "Context information:\n{context}"
question_template = "Question: {question}"

# Compose them
full_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", f"{context_template}\n\n{question_template}"),
    ]
)

chain = full_template | llm
result = chain.invoke(
    {
        "domain": "biology",
        "context": "Photosynthesis is the process by which plants convert light into energy.",
        "question": "What is the primary product of photosynthesis?",
    }
)

print(result.content)

## 9. Templates with Advanced Parameters

Combine prompt engineering with API parameter tuning.

In [ ]:
# Different models for different tasks
analytical_llm = ChatAmazonNova(
    model="nova-pro-v1", temperature=0.2, reasoning_effort="high", max_tokens=200
)

creative_llm = ChatAmazonNova(
    model="nova-pro-v1", temperature=0.9, top_p=0.95, max_tokens=200
)

# Same template, different parameters
template = ChatPromptTemplate.from_template("Write about {topic}")

analytical_chain = template | analytical_llm
creative_chain = template | creative_llm

topic = "the future of AI"

print("Analytical (low temp, high reasoning):")
print(analytical_chain.invoke({"topic": topic}).content)

print("\nCreative (high temp, high top-p):")
print(creative_chain.invoke({"topic": topic}).content)

## Summary

**Prompt Template Patterns:**

| Pattern | Use Case | Benefits |
|---------|----------|----------|
| Simple Template | Basic variable substitution | Easy to use, readable |
| Chat Template | Role-based messaging | Clear conversation structure |
| Few-Shot | Provide examples | Guides model behavior |
| Message Placeholder | Inject chat history | Maintains conversation context |
| Multi-Variable | Complex prompts | Flexible, reusable |
| Partial Variables | Pre-filled templates | Reduces repetition |
| Dynamic Selection | Context-aware examples | Intelligent prompt adaptation |
| Composition | Build from components | Modular, maintainable |

**Best Practices:**
- **Be Specific**: Clear variables names and instructions
- **Provide Examples**: Few-shot learning improves accuracy
- **Use Structure**: System/Human/AI roles for clarity
- **Test Variations**: Try different phrasings
- **Compose Reusably**: Build libraries of template components
- **Version Control**: Track prompt changes like code